In [1]:
import requests
from collections import Counter
from web3 import Web3, HTTPProvider
connection = Web3(HTTPProvider('https://mainnet.infura.io/v3/6873c06628bc4c50af981c610f2f0fc1'))
current_block = connection.eth.blockNumber
print ("Latest Ethereum block number", current_block)
import datetime as dt
import statistics
import time
import numpy as np

Latest Ethereum block number 15149652


In [5]:
number_of_blocks = 1 #ca 5400 blocks per day so 5400 * 30 = 162'000 blocks per month
starting_block = current_block - number_of_blocks
min_wages = 0.3

candidates_dict = {}
most_common_sender_per_address = np.load('./data/most_common_sender_per_address.npy')
previous_addresses = [x['address'] for x in most_common_sender_per_address]


for i in range(number_of_blocks):
    # print(i, " of ", number_of_blocks)
    block_number = starting_block - i
    block = connection.eth.get_block(block_number)
    hex_transaction_hashes = block["transactions"]

    for i in hex_transaction_hashes:
        transaction_hash = i.hex() 
        transaction = connection.eth.get_transaction(transaction_hash)
        value = transaction["value"] / (10**18)
        receiver = transaction["to"] 
        if (value > min_wages and receiver not in previous_addresses):
            if (receiver in candidates_dict):
                list = candidates_dict[receiver]
                list.append(value)
                candidates_dict[receiver] = list
            else:
                candidates_dict[receiver] = [value]

# candidates_dict

In [ ]:
all_info_for_addresses = []
transactions_from_most_common_sender_for_addresses = []

counter = len(candidates_dict)
updates = 0
std_avg_ratio = 0.1

for address in candidates_dict.keys():
    print(updates, " of ", counter)
    updates += 1
    try:
        url = f"https://api.covalenthq.com/v1/1/address/{address}/transactions_v2/?quote-currency=USD&page-size=150&key=ckey_06de8b133f2948eb88f41f10539"
        res = requests.get(url).json()

        transactions = res["data"]["items"]
        transactions_cleaned = []
        sender_addresses = []

        num_transaction_in_22 = len([1 for x in transactions if x["block_signed_at"][0:4] == "2022"])
        if (num_transaction_in_22 > 100):
            continue

        for trans in transactions:
            if(trans["value_quote"] is not None and trans["value_quote"] > 100 and trans["from_address"] != address):
                sender_addresses.append(trans["from_address"])
                transactions_cleaned.append({"timestamp": trans["block_signed_at"], "from": trans["from_address"], "from_label": trans["from_address_label"], "value": float(trans["value"]) / (10**18), "valueUSD": trans["value_quote"]})

        all_info_for_addresses.append({"address": address, "data": transactions_cleaned})

        c = Counter(sender_addresses)
        most_common_address = c.most_common(1)[0][0]

        transactions_from_most_common_sender = []
        year_months = []
        # transactions_cleaned.sort(key=lambda x: time.mktime(dt.datetime(int(x['timestamp'][0:4]),
        #         int(x['timestamp'][5:7]),
        #         int(x['timestamp'][8:10]),
        #         int(x['timestamp'][11:13]),
        #         int(x['timestamp'][14:16])).timetuple()))
        transactions_cleaned.sort(key=lambda x: int(x[0:4] + x[5:7] + x[8:10] + x[11:13] + x[14:16]))
        print('finally sorted')
        last_tx = transactions_cleaned[0]
        time_gaps = []
        for i, trans in enumerate(transactions_cleaned):
            if (trans["from"] == most_common_address):
                transactions_from_most_common_sender.append(trans)
                year_months.append(trans["timestamp"][0:7])
                print(last_tx['timestamp'])
                date1 = dt.datetime(int(last_tx['timestamp'][0:4]),
                        int(last_tx['timestamp'][5:7]),
                        int(last_tx['timestamp'][8:10]),
                        int(last_tx['timestamp'][11:13]),
                        int(last_tx['timestamp'][14:16]))
                date2 = dt.datetime(int(trans['timestamp'][0:4]),
                        int(trans['timestamp'][5:7]),
                        int(trans['timestamp'][8:10]),
                        int(trans['timestamp'][11:13]),
                        int(trans['timestamp'][14:16]))
                time_gaps.append((date2-date1).total_seconds())
                last_tx = trans
        avg = statistics.mean(time_gaps)
        std = statistics.stdev(time_gaps)
        print(f'Avg: {avg}, std: {std}')

        if (len(set(year_months)) >= 10 & avg > 0 and std / avg < std_avg_ratio):
            transactions_from_most_common_sender_for_addresses.append({"address": address, "data": transactions_from_most_common_sender})
            most_common_sender_per_address.append({"address": address, "payer_data": c.most_common(1)})
        elif not (avg > 0 and std / avg < 0.1):
            print("Address", address, "is too different from the average")


        if (counter == 0):
            break
        else:
            print(counter)
    except:
        print("error came up - continuing")
        continue

In [4]:
most_common_sender_per_address

[]

In [10]:
np.save('./data/most_common_sender_per_address', most_common_sender_per_address)
np.save('./data/transactions_from_most_common_sender_for_addresses', transactions_from_most_common_sender_for_addresses)
#most_common_sender_per_address = np.load('./data/most_common_sender_per_address.npz')
#transactions_from_most_common_sender_for_addresses = np.load('./data/transactions_from_most_common_sender_for_addresses.npz')